## 데이터 전처리

In [ ]:
import pandas as pd

file_path = '/content/data_Inspection - 시트12 (1).csv'


# CSV 파일 읽기
df = pd.read_csv(file_path, encoding='utf-8')

# 데이터 확인
print(df.head())

           Question                                             Answer
0            안녕하세요?  안녕하세요!저는 'Coala'입니다. 여러분의 파이썬 학습 도우미에요. Meta의 ...
1          당신은 뭐예요?  안녕하세요!저는 'Coala'입니다. 여러분의 파이썬 학습 도우미에요. Meta의 ...
2         코알라가 뭔가요?  안녕하세요!저는 'Coala'입니다. 여러분의 파이썬 학습 도우미에요. Meta의 ...
3  코알라는 어떻게 만들어졌나요?  안녕하세요!저는 'Coala'입니다. 여러분의 파이썬 학습 도우미에요. Meta의 ...
4             누구세요?  안녕하세요!저는 'Coala'입니다. 여러분의 파이썬 학습 도우미에요. Meta의 ...


In [ ]:
# import pandas as pd
import json

# 먼저 모든 행을 'qas' 형식으로 변환
qas_list = df.apply(lambda row: {'qas': [{'question': row['Question'], 'answers': [{'text': row['Answer']}]}]}, axis=1).tolist()

# 변환된 데이터를 JSON 파일로 저장
json_file_path = '/content/final-data.json'
with open(json_file_path, 'w', encoding='utf-8') as file:
    json.dump(qas_list, file, ensure_ascii=False)


# 저장된 JSON 파일 경로 반환
json_file_path
#길이
len(qas_list)

960

In [ ]:
import json

# JSON 파일을 읽습니다.
with open('/content/final-data.json', 'r') as f:
    dev_data = json.load(f)

# dev_data를 정제하여 refined_dict 생성
refined_dict = {}
for original_dict in dev_data:
    for entry in original_dict['qas']:
        question = entry['question']
        answers = [ans['text'] for ans in entry['answers']][0]
        refined_dict[question] = answers

#print(refined_dict)


In [ ]:
final_prompt_list = []

for idx, (question, answer) in enumerate(refined_dict.items()):
    prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: {question} ### Response: {answer}"
    #print(idx, prompt)
    prompt_dict = {}
    prompt_dict['text'] = prompt
    final_prompt_list.append(prompt_dict)

In [ ]:
final_prompt_list

## Json 파일 형태로 저장 및 다운로드

In [ ]:
with open('/content/final-data.json', 'w', encoding='utf-8') as f:
    f.write("[\n")
    for i, item in enumerate(final_prompt_list):
        json_str = json.dumps(item, ensure_ascii=False)
        f.write(json_str)
        if i < len(final_prompt_list) - 1:  # 마지막 원소가 아니라면 콤마를 추가
            f.write(",\n")
        else:  # 마지막 원소라면 콤마를 추가하지 않음
            f.write("\n")
    f.write("]\n")

# fine-tuning

In [ ]:
!nvidia-smi

Sun Jan 28 10:19:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## autotrain-advanced 모듈 불러오기

In [ ]:
!pip install -q autotrain-advanced

## Torch update

In [ ]:
!autotrain setup --update-torch

> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


## 허깅페이스 토큰 

In [ ]:
!pip install -q transformers accelerate sentencepiece
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 49, in main
    service.run()
  File "/usr/local/lib/python3.

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@main accelerate

In [ ]:
!autotrain llm --train \
    --project-name "llama2-finetuning" \
    --model "TinyPixel/CodeLlama-7B-Python-bf16-sharded" \
    --data-path "aaa" \
    --text-column "text" \
    --peft \
    --quantization "int4" \
    --lr 5e-4 \
    --batch-size 4 \
    --epochs 5 \
    --trainer sft \
    --model_max_length 512 \
    --merge-adapter

> INFO    Running LLM
> INFO    Params: Namespace(version=False, text_column='text', rejected_text_column='rejected', prompt_text_column='prompt', model_ref=None, warmup_ratio=0.1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, add_eos_token=False, block_size=-1, peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, mixed_precision=None, quantization='int4', model_max_length=512, trainer='sft', target_modules=None, merge_adapter=True, use_flash_attention_2=False, dpo_beta=0.1, apply_chat_template=False, padding=None, train=True, deploy=False, inference=False, username=None, backend='local-cli', token=None, repo_id=None, push_to_hub=False, model='TinyPixel/CodeLlama-7B-Python-bf16-sharded', project_name='llama2-finetuning', seed=42, epochs=5, gradient_accumulation=1, disable_gradient_checkpointing=False, lr=0.0005, log='none', data_pa

In [ ]:
import zipfile
import shutil
from google.colab import files

# 압축할 폴더 이름
#folder_name = "llama2-korquad-finetuning"    # Data Augmentation 적용 x
folder_name = "llama2-finetuning-da"  # Data Augmentation 적용 o

# 생성될 ZIP 파일 이름
#zip_file_name = "llama2-korquad-finetuning.zip"  # Data Augmentation 적용 x
zip_file_name = "llama2-finetuning-da.zip" # Data Augmentation 적용 o

# 폴더를 ZIP 파일로 압축
shutil.make_archive(zip_file_name[:-4], 'zip', folder_name)

# ZIP 파일을 로컬로 다운로드
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>